In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
from models.gm_feature_imp import GMInterp

In [3]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import calinski_harabasz_score
from models.transfersdataset import TransfersDataset
from models.xgboost import Xgboost

In [4]:
gm = GaussianMixture()
td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
td.encode_last_positions()
td.encode()
td.drop()

In [5]:
m = Xgboost(data = td.data)
m.train_test_split()
m.scale()

In [6]:
td.data

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,...,posLWB,posRAMF,posRWF,posLWF,posLW,posRW,posGK,posRWB,posDMF,posCB
0,Antoine Griezmann,28,0.0,130.0,Spain,ES1,Atlético de Madrid,Spain,ES1,FC Barcelona,...,0,0,0,0,0,0,0,0,0,0
1,João Félix,19,0.0,70.0,Portugal,PO1,SL Benfica,Spain,ES1,Atlético de Madrid,...,0,0,0,0,0,0,0,0,0,0
2,Harry Maguire,26,0.0,50.0,England,GB1,Leicester City,England,GB1,Manchester United,...,0,0,0,0,0,0,0,0,0,0
3,Frenkie de Jong,22,0.0,85.0,Netherlands,NL1,Ajax Amsterdam,Spain,ES1,FC Barcelona,...,0,0,0,0,0,0,0,0,0,0
4,Matthijs de Ligt,19,0.0,75.0,Netherlands,NL1,Ajax Amsterdam,Italy,IT1,Juventus FC,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2219,Mame Thiam,31,4.0,2.8,Türkiye,TR1,Kayserispor,Türkiye,TR1,Pendikspor,...,0,0,0,0,1,0,0,0,0,0
2220,Yann Gboho,22,4.0,2.5,Belgium,BE1,Cercle Brugge,France,FR1,FC Toulouse,...,0,0,0,1,0,0,0,0,0,0
2221,Kevin Vogt,32,4.0,2.5,Germany,L1,TSG 1899 Hoffenheim,Germany,L1,1.FC Union Berlin,...,0,0,0,0,0,0,0,0,0,1
2222,Daniel Schmidt,31,4.0,1.2,Belgium,BE1,Sint-Truidense VV,Belgium,BE1,KAA Gent,...,0,0,0,0,0,0,1,0,0,0


In [7]:
scores = []
for i in range(2, 11):
    gmm = GaussianMixture(n_components=i, covariance_type='diag', random_state=0)
    gmm.fit(m.X)
    y = gmm.fit_predict(m.X)
    scores.append([
        i,
        gmm.bic(m.X),
        gmm.aic(m.X),
        calinski_harabasz_score(m.X,gmm.predict(m.X))
    ])

scores = pd.DataFrame(scores, columns=['n_clusters', 'bic', 'aic', 'calinski_harabasz_score'])
scores
fig = make_subplots(specs=[[{"secondary_y": True}]])
for metric in scores.columns[1:]:
    if metric == 'calinski_harabasz_score':
        fig.add_trace(go.Scatter(x=scores.n_clusters, y=scores[metric], name=metric), secondary_y=True)
    else:
        fig.add_trace(go.Scatter(x=scores.n_clusters, y=scores[metric], name=metric))
fig.update_layout(height = 500, width = 1000, title = 'Elbow method', xaxis = dict(title_text='Number of clusters'), yaxis = dict(title_text='Scores'))
fig.show()

In [65]:
gmm = GaussianMixture(n_components=9, covariance_type='diag', random_state=0)
gmm.fit(m.X)
labels = gmm.predict(m.X)

In [66]:
player_info_cols = ['name', 'age', 'season', 'country_from', 'league_from', 'club_from',
                            'country_to', 'league_to', 'club_to', 'window', 'fee', 'loan']
preds = m.data
preds['label'] = labels

In [67]:
preds[preds.label == 0]

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,...,posRAMF,posRWF,posLWF,posLW,posRW,posGK,posRWB,posDMF,posCB,label
12,Aaron Wan-Bissaka,21,0.0,35.0,England,GB1,Crystal Palace,England,GB1,Manchester United,...,0,0,0,0,0,0,0,0,0,0
31,Danilo,28,0.0,20.0,England,GB1,Manchester City,Italy,IT1,Juventus FC,...,0,0,0,0,0,0,0,0,0,0
38,Kieran Trippier,28,0.0,35.0,England,GB1,Tottenham Hotspur,Spain,ES1,Atlético de Madrid,...,0,0,0,0,0,0,1,0,0,0
74,Valentino Lazaro,23,0.0,17.0,Germany,L1,Hertha BSC,Italy,IT1,Inter Milan,...,0,0,0,0,0,0,1,0,0,0
100,Quincy Promes,27,0.0,18.0,Spain,ES1,Sevilla FC,Netherlands,NL1,Ajax Amsterdam,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2153,Toni Lato,25,4.0,2.5,Spain,ES1,Valencia CF,Spain,ES1,RCD Mallorca,...,0,0,0,1,1,0,0,0,0,0
2169,Noah Sadiki,18,4.0,2.0,Belgium,BE1,RSC Anderlecht,Belgium,BE1,Union Saint-Gilloise,...,0,0,0,0,0,0,1,0,0,0
2177,Lorenzo Venuti,28,4.0,2.0,Italy,IT1,ACF Fiorentina,Italy,IT1,US Lecce,...,0,0,0,0,0,0,0,0,0,0
2181,Wilson Manafá,29,4.0,2.0,Portugal,PO1,FC Porto,Spain,ES1,Granada CF,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
preds[preds.label == 8].describe()

,age,season,market_value,window,fee,playerid,club_from_elo,club_to_elo,league_from_elo,league_to_elo,...,posRAMF,posRWF,posLWF,posLW,posRW,posGK,posRWB,posDMF,posCB,label
count,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,182.000000,...,182.000000,182.000000,182.000000,182.000000,182.000000,182.0,182.000000,182.0,182.000000,182.0
mean,26.131868,2.016484,6.095192,0.835165,4.069632,262230.302198,1638.032967,1629.752747,1552.320930,1554.731357,...,0.005495,0.016484,0.005495,0.054945,0.049451,0.0,0.373626,0.0,0.010989,0.0
std,3.603892,1.419965,7.697296,0.372055,7.841193,157554.403793,155.538342,135.509676,96.459256,96.933188,...,0.074125,0.127677,0.074125,0.228502,0.217405,0.0,0.485101,0.0,0.104539,0.0
min,18.000000,0.000000,0.500000,0.000000,0.000000,14086.000000,1310.000000,1304.000000,1371.500000,1341.615385,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
25%,23.250000,1.000000,2.000000,1.000000,0.000000,146889.000000,1519.250000,1537.000000,1463.555556,1474.736842,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
50%,27.000000,2.000000,3.500000,1.000000,1.200000,245921.500000,1629.000000,1630.500000,1546.433333,1561.400000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
75%,28.000000,3.000000,7.000000,1.000000,4.437500,340902.250000,1748.500000,1718.500000,1630.900000,1630.900000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.0,0.000000,0.0
max,35.000000,4.000000,60.000000,1.000000,55.000000,816580.000000,2041.000000,1965.000000,1721.066667,1721.066667,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,0.0,1.000000,0.0


In [72]:
def pipeline(cluster):
    td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
    td.encode_last_positions()
    td.encode()
    td.drop()
    m = Xgboost(td.data)
    gmm = GaussianMixture(n_components=9, covariance_type='diag', random_state=0)
    gmm.fit(m.X[m.features])
    labels = gmm.predict(m.X[m.features])
    m.data = m.data.iloc[labels == cluster] 
    m = Xgboost(data = m.data)
    display(m.data)
    m.train_test_split()
    m.scale()
    # m.tune_hp()
    m.train()
    m.inverse_scale()
    m.score(thresh=1.5)
    # display(m.feature_importance())
    m.plot_predictions()
    display(m.top_n_predictions(20))

In [73]:
pipeline(0)

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,...,posLWB,posRAMF,posRWF,posLWF,posLW,posRW,posGK,posRWB,posDMF,posCB
12,Aaron Wan-Bissaka,21,0.0,35.0,England,GB1,Crystal Palace,England,GB1,Manchester United,...,0,0,0,0,0,0,0,0,0,0
31,Danilo,28,0.0,20.0,England,GB1,Manchester City,Italy,IT1,Juventus FC,...,0,0,0,0,0,0,0,0,0,0
38,Kieran Trippier,28,0.0,35.0,England,GB1,Tottenham Hotspur,Spain,ES1,Atlético de Madrid,...,0,0,0,0,0,0,0,1,0,0
74,Valentino Lazaro,23,0.0,17.0,Germany,L1,Hertha BSC,Italy,IT1,Inter Milan,...,0,0,0,0,0,0,0,1,0,0
100,Quincy Promes,27,0.0,18.0,Spain,ES1,Sevilla FC,Netherlands,NL1,Ajax Amsterdam,...,1,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2153,Toni Lato,25,4.0,2.5,Spain,ES1,Valencia CF,Spain,ES1,RCD Mallorca,...,0,0,0,0,1,1,0,0,0,0
2169,Noah Sadiki,18,4.0,2.0,Belgium,BE1,RSC Anderlecht,Belgium,BE1,Union Saint-Gilloise,...,1,0,0,0,0,0,0,1,0,0
2177,Lorenzo Venuti,28,4.0,2.0,Italy,IT1,ACF Fiorentina,Italy,IT1,US Lecce,...,0,0,0,0,0,0,0,0,0,0
2181,Wilson Manafá,29,4.0,2.0,Portugal,PO1,FC Porto,Spain,ES1,Granada CF,...,0,0,0,0,0,0,0,0,0,0


thresh = 1.5
mae: 1.158891151402448
mape: 0.720973182055395
mae_below_thresh: 1.1856162006204778
mape_above_thresh: 0.265869813807315


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error,error_pct
1524,Jimmy Cabot,28,22/23,France,FR1,Angers SCO,France,FR1,RC Lens,1,5.00,3.5,2.00,3.5,3.511732,0.011732,0.003352
187,Vyacheslav Karavaev,24,19/20,Netherlands,NL1,Vitesse Arnhem,Russia,RU1,Zenit St. Petersburg,1,3.00,6.0,8.00,6.0,5.978491,0.021509,0.003585
817,Hidde ter Avest,23,20/21,Italy,IT1,Udinese Calcio,Netherlands,NL1,FC Utrecht,0,1.60,2.0,0.00,2.0,1.906665,0.093335,0.046668
1108,Bart Nieuwkoop,25,21/22,Netherlands,NL1,Feyenoord Rotterdam,Belgium,BE1,Union Saint-Gilloise,1,2.20,2.5,0.00,2.5,2.614151,0.114151,0.045660
2117,Bart Nieuwkoop,27,23/24,Belgium,BE1,Union Saint-Gilloise,Netherlands,NL1,Feyenoord Rotterdam,1,3.00,4.0,2.00,4.0,3.756504,0.243496,0.060874
2072,Bryan Reynolds,22,23/24,Italy,IT1,AS Roma,Belgium,BE1,KVC Westerlo,1,4.00,3.5,3.50,3.5,3.806785,0.306785,0.087653
628,Deyovaisio Zeefuik,22,20/21,Netherlands,NL1,FC Groningen,Germany,L1,Hertha BSC,1,2.70,4.0,4.00,4.0,3.674514,0.325486,0.081371
285,Peter Ankersen,28,19/20,Denmark,DK1,FC Copenhagen,Italy,IT1,Genoa CFC,1,3.00,2.0,1.00,2.0,2.541060,0.541060,0.270530
1347,Héctor Bellerín,27,22/23,England,GB1,Arsenal FC,Spain,ES1,FC Barcelona,1,20.00,8.0,0.00,8.0,8.587776,0.587776,0.073472
2123,Fabiano,23,23/24,Portugal,PO1,SC Braga,Portugal,PO1,Moreirense FC,1,3.00,3.0,0.70,3.0,2.386152,0.613848,0.204616


In [75]:
pipeline(1)

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,...,posLWB,posRAMF,posRWF,posLWF,posLW,posRW,posGK,posRWB,posDMF,posCB
2,Harry Maguire,26,0.0,50.0,England,GB1,Leicester City,England,GB1,Manchester United,...,0,0,0,0,0,0,0,0,0,0
4,Matthijs de Ligt,19,0.0,75.0,Netherlands,NL1,Ajax Amsterdam,Italy,IT1,Juventus FC,...,0,0,0,0,0,0,0,0,0,0
14,Éder Militão,21,0.0,50.0,Portugal,PO1,FC Porto,Spain,ES1,Real Madrid,...,0,0,0,0,0,0,0,0,0,0
21,Konstantinos Manolas,28,0.0,45.0,Italy,IT1,AS Roma,Italy,IT1,SSC Napoli,...,0,0,0,0,0,0,0,0,0,0
34,Jules Koundé,20,0.0,15.0,France,FR1,FC Girondins Bordeaux,Spain,ES1,Sevilla FC,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2201,Danilo D'Ambrosio,34,4.0,1.5,Italy,IT1,Inter Milan,Italy,IT1,AC Monza,...,0,0,0,0,0,0,0,0,0,0
2208,Davide Biraschi,29,4.0,1.3,Italy,IT1,Genoa CFC,Türkiye,TR1,Fatih Karagümrük,...,0,0,0,0,0,0,0,0,0,0
2216,Isak Hien,24,4.0,9.0,Italy,IT1,Hellas Verona,Italy,IT1,Atalanta BC,...,0,0,0,0,0,0,0,0,0,1
2218,Martin Hongla,25,4.0,2.8,Italy,IT1,Hellas Verona,Spain,ES1,Granada CF,...,0,0,0,0,0,0,0,0,1,1


thresh = 1.5
mae: 4.472427929917427
mape: 1.0783094377261857
mae_below_thresh: 3.2776491260528564
mape_above_thresh: 0.5551907560260404


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error,error_pct
1204,Marvin Friedrich,26,21/22,Germany,L1,1.FC Union Berlin,Germany,L1,Borussia Mönchengladbach,0,10.0,8.0,5.50,8.0,7.872845,0.127155,0.015894
1021,Jack Hendry,26,21/22,Scotland,SC1,Celtic FC,Belgium,BE1,KV Oostende,1,3.5,5.0,1.10,5.0,4.738837,0.261163,0.052233
186,Joseph Aidoo,23,19/20,Belgium,BE1,KRC Genk,Spain,ES1,Celta de Vigo,1,4.0,6.5,8.00,6.5,6.819775,0.319775,0.049196
622,Koray Günter,26,20/21,Italy,IT1,Genoa CFC,Italy,IT1,Hellas Verona,1,2.4,5.0,2.40,5.0,4.419912,0.580088,0.116018
258,Mexer,30,19/20,France,FR1,Stade Rennais FC,France,FR1,FC Girondins Bordeaux,1,4.0,3.2,0.00,3.2,3.866117,0.666117,0.208161
2077,Andreaw Gravillon,25,23/24,France,FR1,Stade Reims,Türkiye,TR1,Adana Demirspor,1,4.0,4.7,2.50,4.7,5.474941,0.774941,0.164881
271,Igor Diveev,19,19/20,Russia,RU1,FK Ufa,Russia,RU1,CSKA Moscow,1,3.5,4.5,1.50,4.5,5.285471,0.785471,0.174549
1828,Axel Disasi,25,23/24,France,FR1,AS Monaco,England,GB1,Chelsea FC,1,30.0,42.0,45.00,42.0,42.819675,0.819675,0.019516
1858,Nathan Collins,22,23/24,England,GB1,Wolverhampton Wanderers,England,GB1,Brentford FC,1,22.0,25.0,26.85,25.0,24.109293,0.890707,0.035628
1435,Jhon Lucumí,24,22/23,Belgium,BE1,KRC Genk,Italy,IT1,Bologna FC 1909,1,9.0,12.0,8.00,12.0,13.136390,1.136390,0.094699


In [76]:
pipeline(3)

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,...,posLWB,posRAMF,posRWF,posLWF,posLW,posRW,posGK,posRWB,posDMF,posCB
0,Antoine Griezmann,28,0.0,130.0,Spain,ES1,Atlético de Madrid,Spain,ES1,FC Barcelona,...,0,0,0,0,0,0,0,0,0,0
1,João Félix,19,0.0,70.0,Portugal,PO1,SL Benfica,Spain,ES1,Atlético de Madrid,...,0,0,0,0,0,0,0,0,0,0
7,Romelu Lukaku,26,0.0,75.0,England,GB1,Manchester United,Italy,IT1,Inter Milan,...,0,0,0,0,0,0,0,0,0,0
10,Luka Jović,21,0.0,60.0,Germany,L1,Eintracht Frankfurt,Spain,ES1,Real Madrid,...,0,0,0,0,0,0,0,0,0,0
13,Sébastien Haller,25,0.0,40.0,Germany,L1,Eintracht Frankfurt,England,GB1,West Ham United,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2213,Anderson Silva,25,4.0,1.2,Portugal,PO1,Vitória Guimarães SC,Türkiye,TR1,Alanyaspor,...,0,0,0,0,0,0,0,0,0,0
2215,Mohamed-Ali Cho,19,4.0,8.0,Spain,ES1,Real Sociedad,France,FR1,OGC Nice,...,0,0,1,0,0,1,0,0,0,0
2219,Mame Thiam,31,4.0,2.8,Türkiye,TR1,Kayserispor,Türkiye,TR1,Pendikspor,...,0,0,0,0,1,0,0,0,0,0
2220,Yann Gboho,22,4.0,2.5,Belgium,BE1,Cercle Brugge,France,FR1,FC Toulouse,...,0,0,0,1,0,0,0,0,0,0


thresh = 1.5
mae: 3.0187024687653157
mape: 0.6355330086693584
mae_below_thresh: 1.3580229622977125
mape_above_thresh: 0.440260416290903


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error,error_pct
1761,João Figueiredo,26,22/23,Türkiye,TR1,Gaziantep FK,Türkiye,TR1,Basaksehir FK,0,3.0,2.8,0.65,2.8,2.851534,0.051534,0.018405
2002,Ángel Di María,35,23/24,Italy,IT1,Juventus FC,Portugal,PO1,SL Benfica,1,8.0,5.0,0.00,5.0,4.947858,0.052142,0.010428
1086,Mame Thiam,28,21/22,Türkiye,TR1,Fenerbahce,Türkiye,TR1,Kayserispor,1,2.5,3.0,1.50,3.0,3.088655,0.088655,0.029552
1743,Bamba Dieng,22,22/23,France,FR1,Olympique Marseille,France,FR1,FC Lorient,0,10.0,8.0,7.00,8.0,8.099588,0.099588,0.012449
237,Stefano Okaka,30,19/20,England,GB1,Watford FC,Italy,IT1,Udinese Calcio,1,5.0,3.2,0.00,3.2,3.345984,0.145984,0.045620
1537,David Okereke,24,22/23,Belgium,BE1,Club Brugge KV,Italy,IT1,US Cremonese,1,4.5,4.0,0.00,4.0,4.147227,0.147227,0.036807
25,Moise Kean,19,19/20,Italy,IT1,Juventus FC,England,GB1,Everton FC,1,40.0,25.0,27.50,25.0,24.821327,0.178673,0.007147
1690,Islam Slimani,34,22/23,Portugal,PO1,Sporting CP,France,FR1,Stade Brestois 29,1,1.5,1.5,0.00,1.5,1.699001,0.199001,0.132667
2028,Raúl Jiménez,32,23/24,England,GB1,Wolverhampton Wanderers,England,GB1,Fulham FC,1,6.0,6.0,6.40,6.0,5.776408,0.223592,0.037265
83,Dodi Lukébakio,21,19/20,England,GB1,Watford FC,Germany,L1,Hertha BSC,1,18.0,19.0,20.00,19.0,18.750420,0.249580,0.013136


In [64]:
pipeline(3)

,name,age,season,market_value,country_from,league_from,club_from,country_to,league_to,club_to,...,posLWB,posRAMF,posRWF,posLWF,posLW,posRW,posGK,posRWB,posDMF,posCB
35,Jasper Cillessen,30,0.0,18.0,Spain,ES1,FC Barcelona,Spain,ES1,Valencia CF,...,0,0,0,0,0,0,1,0,0,0
58,Neto,29,0.0,18.0,Spain,ES1,Valencia CF,Spain,ES1,FC Barcelona,...,0,0,0,0,0,0,1,0,0,0
71,Pau López,24,0.0,20.0,Spain,ES1,Real Betis Balompié,Italy,IT1,AS Roma,...,0,0,0,0,0,0,1,0,0,0
87,Emil Audero,22,0.0,15.0,Italy,IT1,Juventus FC,Italy,IT1,UC Sampdoria,...,0,0,0,0,0,0,1,0,0,0
109,Benjamin Lecomte,28,0.0,17.0,France,FR1,Montpellier HSC,France,FR1,AS Monaco,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2151,Maxime Dupé,30,4.0,2.5,France,FR1,FC Toulouse,Belgium,BE1,RSC Anderlecht,...,0,0,0,0,0,0,1,0,0,0
2152,Finn Dahmen,25,4.0,2.5,Germany,L1,1.FSV Mainz 05,Germany,L1,FC Augsburg,...,0,0,0,0,0,0,1,0,0,0
2171,Timon Wellenreuther,27,4.0,2.0,Belgium,BE1,RSC Anderlecht,Netherlands,NL1,Feyenoord Rotterdam,...,0,0,0,0,0,0,1,0,0,0
2192,Marco Sportiello,31,4.0,1.5,Italy,IT1,Atalanta BC,Italy,IT1,AC Milan,...,0,0,0,0,0,0,1,0,0,0


thresh = 1.5
mae: 5.412350762517829
mape: 1.4739041857769397
mae_below_thresh: 3.016346130371094
mape_above_thresh: 0.6591836762155511


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,marketval,marketval_0,fee,marketval_0,marketval_0_pred,error,error_pct
1649,Paul Nardi,28,22/23,France,FR1,FC Lorient,Belgium,BE1,KAA Gent,1,2.000,3.0,0.15,3.0,3.338866,0.338866,0.112955
223,Jesse Joronen,26,19/20,Denmark,DK1,FC Copenhagen,Italy,IT1,Brescia Calcio,1,2.500,4.0,5.00,4.0,4.481149,0.481149,0.120287
324,Alfred Gomis,25,19/20,Italy,IT1,SPAL,France,FR1,Dijon FCO,1,2.000,2.4,1.00,2.4,3.416368,1.016368,0.423487
880,Mike Maignan,25,21/22,France,FR1,LOSC Lille,Italy,IT1,AC Milan,1,25.000,35.0,16.40,35.0,36.213963,1.213963,0.034685
685,Jean Butez,25,20/21,Belgium,BE1,Royal Excel Mouscron (-2022),Belgium,BE1,Royal Antwerp FC,1,2.000,2.0,1.00,2.0,3.338866,1.338866,0.669433
678,Thomas Didillon,24,20/21,Belgium,BE1,RSC Anderlecht,Belgium,BE1,Cercle Brugge,1,2.800,1.5,1.30,1.5,2.985152,1.485152,0.990102
1685,Stanislav Kritsyuk,31,22/23,Russia,RU1,Zenit St. Petersburg,Portugal,PO1,Gil Vicente FC,1,1.500,1.2,0.00,1.2,3.205804,2.005804,1.671503
1686,Stanislav Kritsyuk,31,22/23,Russia,RU1,Zenit St. Petersburg,Portugal,PO1,Gil Vicente FC,1,1.500,1.2,0.00,1.2,3.205804,2.005804,1.671503
574,Alexander Schwolow,28,20/21,Germany,L1,SC Freiburg,Germany,L1,Hertha BSC,1,7.000,5.0,7.00,5.0,7.047188,2.047188,0.409438
1087,Frederik Rönnow,28,21/22,Germany,L1,Eintracht Frankfurt,Germany,L1,1.FC Union Berlin,1,2.500,2.0,1.00,2.0,4.531462,2.531462,1.265731


In [ ]:
kms = GMInterp(
	n_components=5,
	ordered_feature_names=m.X.columns.tolist(), 
	feature_importance_method='wcss_min', # or 'unsup2sup'
).fit(m.X)

# A dictionary where the key [0] is the cluster label, and [:10] will refer to the first 10 most important features
kms.feature_importances_[0]

TypeError: super(type, obj): obj must be an instance or subtype of type